In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

from vorflow import ConceptualMesh, MeshGenerator, VoronoiTessellator
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString, box

In [ ]:

domain = box(0, 0, 200, 150)
poly1 = box(50, 50, 75, 75)
# generate sine wave curve for river
river_coords = [(x, 75 + 20 * np.sin(0.1 * x)) for x in range(-10, 220, 10)]
river_coords = LineString(river_coords)
well_point = Point(25, 25)
fault_line = LineString([(100, 0), (100, 150)])

In [ ]:
# 1. Setup Blueprint
blueprint = ConceptualMesh(crs="EPSG:3857")
blueprint.add_polygon(domain, zone_id=1, resolution=200)
blueprint.add_polygon(poly1, zone_id=2, resolution=5, z_order=1) 
blueprint.add_point(well_point, point_id="Well-A", resolution=100)
blueprint.add_line(fault_line, line_id="Fault-1", resolution=5, 
                   is_barrier=True)
blueprint.add_line(river_coords, line_id="riv-1", resolution=3, 
                   is_barrier=False)

clean_polys, clean_lines, clean_pts = blueprint.generate()

# 2. Mesh Generation
mesher = MeshGenerator()
mesher.generate(clean_polys, clean_lines, clean_pts)

# 3. Voronoi Conversion
tessellator = VoronoiTessellator(mesher, blueprint)
grid_gdf = tessellator.generate()

# 4. Output
#grid_gdf.to_file("mf6_grid.shp")

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,15))
ax.set_aspect('equal')
# plot domain
ax.plot(*domain.exterior.xy,marker='.', color='black')
# plot poly1
ax.plot(*poly1.exterior.xy, color='blue',lw=1,ls='--')
ax.plot(*river_coords.xy, color='cyan',lw=1,ls='--')
ax.plot(*fault_line.xy, color='r',lw=1,ls='--')
ax.scatter(*well_point.xy, color='green',s=100,marker='x')
grid_gdf.plot(ax=ax)